In [1]:
import pandas as pd
import numpy as np

## Reading in Data

In [2]:
event_feature_map = pd.read_csv('~/git/GeorgiaTech/cse6250/903178639-vla6-hw1/data/train/event_feature_map.csv')
events = pd.read_csv('~/git/GeorgiaTech/cse6250/903178639-vla6-hw1/data/train/events.csv')
mortality = pd.read_csv('~/git/GeorgiaTech/cse6250/903178639-vla6-hw1/data/train/mortality_events.csv')

## Exploring Data

In [3]:
events.head()

,patient_id,event_id,event_description,timestamp,value
0,12,DIAG440649,Primary malignant neoplasm of head of pancreas,2011-12-06,1.0
1,12,DIAG201070,Cholelithiasis AND cholecystitis without obstr...,2011-12-06,1.0
2,12,DIAG321462,Cardiac complication,2011-12-06,1.0
3,12,DIAG321042,Cardiac arrest,2011-12-06,1.0
4,12,DIAG435141,Hemorrhage AND/OR hematoma complicating procedure,2011-12-06,1.0


In [4]:
events.shape

(740066, 5)

In [5]:
event_feature_map.head()

,idx,event_id
0,1,DIAG132397
1,2,DIAG132408
2,3,DIAG132446
3,4,DIAG132583
4,5,DIAG132643


In [6]:
mortality.head()

,patient_id,timestamp,label
0,19,2014-03-04,1
1,12,2011-12-19,1
2,41,2014-02-15,1
3,106,2015-08-11,1
4,112,2011-04-23,1


In [7]:
mortality.shape

(500, 3)

In [8]:
mortality.label.value_counts()

1    500
Name: label, dtype: int64

In [9]:
event_feature_map.idx.value_counts().value_counts()

1    3188
Name: idx, dtype: int64

## Descriptive Stats

In [10]:
df = events.merge(mortality[['patient_id', 'label']], how='left', on='patient_id')
df.rename(index=str, columns={'label': 'is_deceased'}, inplace=True)
df.is_deceased.fillna(0, inplace=True)
df.is_deceased = df.is_deceased.astype(bool)

In [11]:
df.head()

,patient_id,event_id,event_description,timestamp,value,is_deceased
0,12,DIAG440649,Primary malignant neoplasm of head of pancreas,2011-12-06,1.0,True
1,12,DIAG201070,Cholelithiasis AND cholecystitis without obstr...,2011-12-06,1.0,True
2,12,DIAG321462,Cardiac complication,2011-12-06,1.0,True
3,12,DIAG321042,Cardiac arrest,2011-12-06,1.0,True
4,12,DIAG435141,Hemorrhage AND/OR hematoma complicating procedure,2011-12-06,1.0,True


In [12]:
df.is_deceased.value_counts()

True     491007
False    249059
Name: is_deceased, dtype: int64

In [13]:
df.head()

,patient_id,event_id,event_description,timestamp,value,is_deceased
0,12,DIAG440649,Primary malignant neoplasm of head of pancreas,2011-12-06,1.0,True
1,12,DIAG201070,Cholelithiasis AND cholecystitis without obstr...,2011-12-06,1.0,True
2,12,DIAG321462,Cardiac complication,2011-12-06,1.0,True
3,12,DIAG321042,Cardiac arrest,2011-12-06,1.0,True
4,12,DIAG435141,Hemorrhage AND/OR hematoma complicating procedure,2011-12-06,1.0,True


### Event Counts by Deceased vs Alive Patients

In [14]:
event_counts = df.groupby(['patient_id', 'is_deceased']).size().reset_index(name='number_events')

In [15]:
event_counts.head()

,patient_id,is_deceased,number_events
0,12,True,868
1,19,True,177
2,41,True,1092
3,80,False,185
4,99,False,238


**Answer: Event Counts by Deceased Status**

In [16]:
event_counts_results = event_counts.groupby(['is_deceased']).agg(['mean', 'min', 'max'])['number_events']

In [17]:
event_counts_results

,mean,min,max
is_deceased,,,
False,498.118,1,12627
True,982.014,1,8635


In [18]:
avg_dead_event_count = event_counts_results.loc[True]['mean']
max_dead_event_count = event_counts_results.loc[True]['max']
min_dead_event_count = event_counts_results.loc[True]['min']
avg_alive_event_count = event_counts_results.loc[False]['mean']
max_alive_event_count = event_counts_results.loc[False]['max']
min_alive_event_count = event_counts_results.loc[False]['min']

In [19]:
avg_dead_event_count, max_dead_event_count, min_dead_event_count, avg_alive_event_count, max_alive_event_count, min_alive_event_count

(982.014, 8635.0, 1.0, 498.118, 12627.0, 1.0)

### Encounters by Deceased vs Alive Patients

In [20]:
events.head()

,patient_id,event_id,event_description,timestamp,value
0,12,DIAG440649,Primary malignant neoplasm of head of pancreas,2011-12-06,1.0
1,12,DIAG201070,Cholelithiasis AND cholecystitis without obstr...,2011-12-06,1.0
2,12,DIAG321462,Cardiac complication,2011-12-06,1.0
3,12,DIAG321042,Cardiac arrest,2011-12-06,1.0
4,12,DIAG435141,Hemorrhage AND/OR hematoma complicating procedure,2011-12-06,1.0


In [21]:
encounters = events[['patient_id', 'timestamp']].drop_duplicates()
encounters.sort_values(['patient_id', 'timestamp'], inplace=True)
encounters['encounter_id'] = [i for i in range(encounters.shape[0])]
encounters.set_index('encounter_id', inplace=True)

In [22]:
encounters.head()

,patient_id,timestamp
encounter_id,,
0,12,2011-12-04
1,12,2011-12-06
2,12,2011-12-07
3,12,2011-12-08
4,12,2011-12-09


In [23]:
## Counting number of encounters and splitting by deceased status
encounter_counts = encounters.groupby(['patient_id']).size().reset_index(name='number_encounters')
encounter_counts = encounter_counts.merge(mortality[['patient_id', 'label']], how='left', on='patient_id')
encounter_counts.rename(index=str, columns={'label': 'is_deceased'}, inplace=True)
encounter_counts.is_deceased.fillna(0, inplace=True)
encounter_counts.is_deceased = encounter_counts.is_deceased.astype(bool)

**Answer: Encounter Counts by deceased patients**

In [24]:
encounter_counts_results = encounter_counts.groupby(['is_deceased']).agg(['mean', 'max', 'min'])['number_encounters']

In [25]:
encounter_counts_results

,mean,max,min
is_deceased,,,
False,15.452,391,1
True,23.038,203,1


In [26]:
avg_dead_encounter_count = encounter_counts_results.loc[True]['mean']
max_dead_encounter_count = encounter_counts_results.loc[True]['max']
min_dead_encounter_count = encounter_counts_results.loc[True]['min']
avg_alive_encounter_count = encounter_counts_results.loc[False]['mean']
max_alive_encounter_count = encounter_counts_results.loc[False]['max']
min_alive_encounter_count = encounter_counts_results.loc[False]['min']

## Record Length by Deceased Status

In [27]:
events.head()

,patient_id,event_id,event_description,timestamp,value
0,12,DIAG440649,Primary malignant neoplasm of head of pancreas,2011-12-06,1.0
1,12,DIAG201070,Cholelithiasis AND cholecystitis without obstr...,2011-12-06,1.0
2,12,DIAG321462,Cardiac complication,2011-12-06,1.0
3,12,DIAG321042,Cardiac arrest,2011-12-06,1.0
4,12,DIAG435141,Hemorrhage AND/OR hematoma complicating procedure,2011-12-06,1.0


In [28]:
record_length = events[['patient_id', 'timestamp']].groupby('patient_id').agg(['min', 'max'])
record_length.columns = ['min_timestamp', 'max_timestamp']
record_length['record_length'] = (pd.to_datetime(record_length.max_timestamp) - pd.to_datetime(record_length.min_timestamp)).dt.days
record_length.reset_index(inplace=True)

In [29]:
record_length.head()

,patient_id,min_timestamp,max_timestamp,record_length
0,12,2011-12-04,2011-12-19,15
1,19,2013-02-19,2013-02-23,4
2,41,2013-05-13,2013-06-25,43
3,80,2015-01-19,2015-01-27,8
4,99,2010-04-25,2013-10-13,1267


In [30]:
## Splitting by deceased status
record_length = record_length.merge(mortality[['patient_id', 'label']], how='left', on='patient_id')
record_length.rename(index=str, columns={'label': 'is_deceased'}, inplace=True)
record_length.is_deceased.fillna(0, inplace=True)
record_length.is_deceased = record_length.is_deceased.astype(bool)
record_length.record_length = record_length.record_length.astype(int)

In [31]:
record_length.head()

,patient_id,min_timestamp,max_timestamp,record_length,is_deceased
0,12,2011-12-04,2011-12-19,15,True
1,19,2013-02-19,2013-02-23,4,True
2,41,2013-05-13,2013-06-25,43,True
3,80,2015-01-19,2015-01-27,8,False
4,99,2010-04-25,2013-10-13,1267,False


**Answer: Record Length by Deceased Status**

In [32]:
record_length_results = record_length.groupby(['is_deceased']).agg(['mean', 'max', 'min'])['record_length']

In [33]:
record_length_results

,mean,max,min
is_deceased,,,
False,159.200,2914,0
True,127.532,1972,0


In [34]:
avg_dead_rec_len = record_length_results.loc[True]['mean']
max_dead_rec_len = record_length_results.loc[True]['max']
min_dead_rec_len = record_length_results.loc[True]['min']
avg_alive_rec_len = record_length_results.loc[False]['mean']
max_alive_rec_len = record_length_results.loc[False]['max']
min_alive_rec_len = record_length_results.loc[False]['min']

## Calculating Index Date

In [35]:
events.head()

,patient_id,event_id,event_description,timestamp,value
0,12,DIAG440649,Primary malignant neoplasm of head of pancreas,2011-12-06,1.0
1,12,DIAG201070,Cholelithiasis AND cholecystitis without obstr...,2011-12-06,1.0
2,12,DIAG321462,Cardiac complication,2011-12-06,1.0
3,12,DIAG321042,Cardiac arrest,2011-12-06,1.0
4,12,DIAG435141,Hemorrhage AND/OR hematoma complicating procedure,2011-12-06,1.0


In [36]:
indx_date = events[['patient_id', 'timestamp']].groupby(['patient_id'])\
                                               .agg({'timestamp': 'max'})\
                                               .rename(columns={'timestamp': 'last_event_date'})\
                                               .reset_index()

In [37]:
indx_date = indx_date.merge(mortality[['patient_id', 'label', 'timestamp']].rename(columns={'timestamp': 'date_of_death'}), how='left', on='patient_id')
indx_date.rename(index=str, columns={'label': 'is_deceased'}, inplace=True)
indx_date.is_deceased.fillna(0, inplace=True)
indx_date.is_deceased = indx_date.is_deceased.astype(bool)
indx_date.last_event_date = pd.to_datetime(indx_date.last_event_date)
indx_date.date_of_death = pd.to_datetime(indx_date.date_of_death)
indx_date['date_of_death_30_days_prior'] = indx_date.date_of_death - pd.to_timedelta(30, unit='d')
indx_date['indx_date'] = indx_date.date_of_death_30_days_prior.combine_first(indx_date.last_event_date)
indx_date = indx_date[['patient_id', 'indx_date']]

In [38]:
indx_date.head()

,patient_id,indx_date
0,12,2011-11-19
1,19,2014-02-02
2,41,2014-01-16
3,80,2015-01-27
4,99,2013-10-13


In [39]:
indx_date.patient_id.value_counts().value_counts()

1    1000
Name: patient_id, dtype: int64

## Filter Events

In [40]:
events.head()

,patient_id,event_id,event_description,timestamp,value
0,12,DIAG440649,Primary malignant neoplasm of head of pancreas,2011-12-06,1.0
1,12,DIAG201070,Cholelithiasis AND cholecystitis without obstr...,2011-12-06,1.0
2,12,DIAG321462,Cardiac complication,2011-12-06,1.0
3,12,DIAG321042,Cardiac arrest,2011-12-06,1.0
4,12,DIAG435141,Hemorrhage AND/OR hematoma complicating procedure,2011-12-06,1.0


In [41]:
indx_date.head()

,patient_id,indx_date
0,12,2011-11-19
1,19,2014-02-02
2,41,2014-01-16
3,80,2015-01-27
4,99,2013-10-13


In [42]:
indx_date['observation_window_begin'] = indx_date.indx_date - pd.to_timedelta(2000, unit='d')
indx_date['prediction_window_end'] = indx_date.indx_date + pd.to_timedelta(30, unit='d')

In [43]:
filtered_events = events.merge(indx_date, how='left', on='patient_id')
filtered_events.timestamp = pd.to_datetime(filtered_events.timestamp)

In [44]:
filtered_events['keep_event'] = (filtered_events.timestamp >= filtered_events.observation_window_begin) &\
                                (filtered_events.timestamp <= filtered_events.prediction_window_end)

In [45]:
filtered_events = filtered_events.loc[filtered_events.keep_event]

In [46]:
filtered_events = filtered_events[['patient_id', 'event_id', 'value']]

In [47]:
filtered_events.head()

,patient_id,event_id,value
0,12,DIAG440649,1.0
1,12,DIAG201070,1.0
2,12,DIAG321462,1.0
3,12,DIAG321042,1.0
4,12,DIAG435141,1.0


## Aggregate Events

In [48]:
filtered_events_df = filtered_events
feature_map_df = event_feature_map

In [49]:
filtered_events_df.head()

,patient_id,event_id,value
0,12,DIAG440649,1.0
1,12,DIAG201070,1.0
2,12,DIAG321462,1.0
3,12,DIAG321042,1.0
4,12,DIAG435141,1.0


In [50]:
feature_map_df.head()

,idx,event_id
0,1,DIAG132397
1,2,DIAG132408
2,3,DIAG132446
3,4,DIAG132583
4,5,DIAG132643


Checking Nulls

In [51]:
filtered_events_df.value.isnull().sum()

71734

In [52]:
filtered_events_df.value.isnull().size

732945

In [53]:
# Need to Remove rows where value is null
events_remove_na = filtered_events_df.loc[filtered_events_df.value.isnull() == False]

In [54]:
events_remove_na = events_remove_na.merge(feature_map_df, how='left', on='event_id')
events_remove_na.rename(columns={'idx': 'event_id_idx'}, inplace=True)
events_remove_na['event_type'] = events_remove_na.event_id.str.replace('[^a-zA-Z]', '')

In [55]:
events_remove_na.head()

,patient_id,event_id,value,event_id_idx,event_type
0,12,DIAG440649,1.0,1658,DIAG
1,12,DIAG201070,1.0,420,DIAG
2,12,DIAG321462,1.0,736,DIAG
3,12,DIAG321042,1.0,729,DIAG
4,12,DIAG435141,1.0,1317,DIAG


In [56]:
events_grouped = events_remove_na[['patient_id', 'event_id_idx', 'event_type', 'value']].groupby(['patient_id', 'event_id_idx', 'event_type'])\
                                                                                        .agg({'value': ['count', 'sum']})
    
# Neat trick to collapse the hierarchical columns returned by groupby to single column
events_grouped.columns = ['_'.join(col).strip() for col in events_grouped.columns.values]

# reset index
events_grouped.reset_index(inplace=True)
    

In [57]:
events_grouped.head()

,patient_id,event_id_idx,event_type,value_count,value_sum
0,12,247,DIAG,1,1.0
1,12,346,DIAG,1,1.0
2,12,420,DIAG,1,1.0
3,12,719,DIAG,1,1.0
4,12,729,DIAG,1,1.0


In [58]:
events_grouped['value'] = np.where(events_grouped['event_type']=='LAB',
                                   events_grouped.value_count,
                                   events_grouped.value_sum)

In [59]:
events_grouped.loc[events_grouped.value == 0]

,patient_id,event_id_idx,event_type,value_count,value_sum,value


In [60]:
aggregated_events = events_grouped[['patient_id', 'event_id_idx', 'value']]
aggregated_events = aggregated_events.rename(columns={'event_id_idx': 'feature_id', 'value': 'feature_value'})

In [61]:
aggregated_events.loc[aggregated_events.feature_value == 0]

,patient_id,feature_id,feature_value


In [62]:
aggregated_events.head()

,patient_id,feature_id,feature_value
0,12,247,1.0
1,12,346,1.0
2,12,420,1.0
3,12,719,1.0
4,12,729,1.0


In [63]:
feature_min_max = aggregated_events[['feature_id', 'feature_value']].groupby(['feature_id']).agg({'feature_value': ['min', 'max']})
feature_min_max.columns = ['_'.join(col).strip() for col in feature_min_max.columns.values]
feature_min_max.reset_index(inplace=True)

In [64]:
feature_min_max.head()

,feature_id,feature_value_min,feature_value_max
0,1,1.0,1.0
1,3,1.0,1.0
2,5,1.0,1.0
3,7,1.0,1.0
4,8,1.0,1.0


In [65]:
aggregated_events = aggregated_events.merge(feature_min_max, on=['feature_id'])

In [66]:
aggregated_events.head()

,patient_id,feature_id,feature_value,feature_value_min,feature_value_max
0,12,247,1.0,1.0,2.0
1,5866,247,1.0,1.0,2.0
2,5962,247,1.0,1.0,2.0
3,7391,247,1.0,1.0,2.0
4,7786,247,1.0,1.0,2.0


In [67]:
aggregated_events['feature_value_normalized'] = (aggregated_events.feature_value) / (aggregated_events.feature_value_max)

In [68]:
aggregated_events = aggregated_events[['patient_id', 'feature_id', 'feature_value_normalized']].rename(columns={'feature_value_normalized': 'feature_value'})

In [69]:
aggregated_events['patient_id'] = aggregated_events.patient_id.astype(float)
aggregated_events['feature_id'] = aggregated_events.feature_id.astype(float)

In [70]:
aggregated_events.head()

,patient_id,feature_id,feature_value
0,12.0,247.0,0.5
1,5866.0,247.0,0.5
2,5962.0,247.0,0.5
3,7391.0,247.0,0.5
4,7786.0,247.0,0.5


In [71]:
# Checking that aggregated_events is normalized properly, min is 0 and max is 1
print(min(aggregated_events.feature_value), max(aggregated_events.feature_value))

0.0018975332068311196 1.0


## Create Features

In [72]:
mortality_df = events[['patient_id']].drop_duplicates().reset_index(drop=True)

In [73]:
mortality_df = mortality_df.merge(mortality[['patient_id', 'label']], how='left', on='patient_id')
mortality_df.rename(index=str, columns={'label': 'is_deceased'}, inplace=True)
mortality_df.is_deceased.fillna(0, inplace=True)

In [74]:
mortality_dict = dict(zip(mortality_df.patient_id, mortality_df.is_deceased))

Next, Patient Features

In [75]:
aggregated_events.sort_values(['patient_id', 'feature_id'], inplace=True)

In [76]:
aggregated_events.head()

,patient_id,feature_id,feature_value
0,12.0,247.0,0.50
8,12.0,346.0,0.50
14,12.0,420.0,1.00
18,12.0,719.0,0.25
306,12.0,729.0,1.00


In [77]:
patient_features = {}
patients = events.patient_id.drop_duplicates()

In [78]:
for p in patients:
    patient_features[p] = []

In [79]:
for index, row in aggregated_events.iterrows():
    patient_features[row.patient_id].append((row.feature_id, row.feature_value))

In [80]:
patients.sort_values(inplace=True)

In [81]:
patient_features[12]

[(247.0, 0.5),
 (346.0, 0.5),
 (420.0, 1.0),
 (719.0, 0.25),
 (729.0, 1.0),
 (736.0, 1.0),
 (1317.0, 0.5),
 (1658.0, 1.0),
 (1832.0, 1.0),
 (2075.0, 0.15),
 (2092.0, 0.2),
 (2193.0, 0.3),
 (2221.0, 0.16666666666666666),
 (2232.0, 0.4),
 (2272.0, 0.058823529411764705),
 (2280.0, 0.3333333333333333),
 (2282.0, 0.041666666666666664),
 (2332.0, 0.09090909090909091),
 (2343.0, 0.10344827586206896),
 (2347.0, 0.06666666666666667),
 (2350.0, 0.04285714285714286),
 (2378.0, 0.3076923076923077),
 (2383.0, 0.125),
 (2400.0, 0.2857142857142857),
 (2414.0, 0.09090909090909091),
 (2447.0, 0.0375),
 (2448.0, 0.010869565217391304),
 (2453.0, 0.0967741935483871),
 (2475.0, 0.029411764705882353),
 (2491.0, 0.1111111111111111),
 (2498.0, 0.20833333333333334),
 (2543.0, 0.05673758865248227),
 (2593.0, 0.18518518518518517),
 (2599.0, 0.043478260869565216),
 (2612.0, 0.2222222222222222),
 (2635.0, 0.027777777777777776),
 (2640.0, 0.1111111111111111),
 (2682.0, 0.13043478260869565),
 (2684.0, 0.0625),
 (268

In [82]:
patients = [1, 5, 3]

In [83]:
patients.sort()

In [84]:
patients

[1, 3, 5]

In [86]:
events.loc[events.value.isnull()]

,patient_id,event_id,event_description,timestamp,value
11168,13621,LAB3005707,Macrocytes [Presence] in Blood,2015-09-17,NaN
11169,13621,LAB3025639,Microcytes [Presence] in Blood,2015-09-17,NaN
11170,13621,LAB3011987,Polychromasia [Presence] in Blood by Light mic...,2015-09-17,NaN
11171,13621,LAB3025616,Target cells [Presence] in Blood by Light micr...,2015-09-17,NaN
11172,13621,LAB3019880,Schistocytes [Presence] in Blood by Light micr...,2015-09-17,NaN
11173,13621,LAB3005854,Burr cells [Presence] in Blood by Light micros...,2015-09-17,NaN
11183,13621,LAB3004809,Neutrophils.band form/100 leukocytes in Blood,2015-09-17,NaN
11186,13621,LAB3006504,Eosinophils/100 leukocytes in Blood,2015-09-17,NaN
11187,13621,LAB3022096,Basophils/100 leukocytes in Blood,2015-09-17,NaN
11189,13621,LAB3002179,Metamyelocytes/100 leukocytes in Blood,2015-09-17,NaN
